# 综述
pLSA:[Probabilistic latent semantic analysis](https://arxiv.org/pdf/1301.6705.pdf)，EM算法，较为简单，容易过拟合（M步存在参数：$p(z_k|d_m)$，根训练样本完全相关）

LDA: 参考[lda-LDA数学八卦](http://bloglxm.oss-cn-beijing.aliyuncs.com/lda-LDA%E6%95%B0%E5%AD%A6%E5%85%AB%E5%8D%A6.pdf ):第45页很形象（与pLSA区别：topic属于一个Dirichlet分布），Gibbs Sample（推导用了一些近似方法）。短文本实际效果并不太好

plsa泛化能力不好，主题分布是确定的，频率派参数估计基于已有数据。

DA数学推理优秀，但是实际用的时候，gibbs sampling公式其实是一个近似正比，效果也没有非常理想。反而plsa简单容易并行化。


# 工具
gensim，百度也开源了LDA工具，LDA不像word2vec，需要去除停用词
https://radimrehurek.com/gensim/models/ldamodel.html

# Ref
[NLP —— 图模型（三）pLSA（Probabilistic latent semantic analysis，概率隐性语义分析）模型](https://www.cnblogs.com/Determined22/p/7237111.html)

实践可以参考：https://www.cnblogs.com/chenbjin/p/5635853.html



In [10]:
from gensim.test.utils import common_texts
from gensim.corpora.dictionary import Dictionary
import gensim.models.ldamulticore
from gensim.models import LdaModel,LdaMulticore

# Create a corpus from a list of texts

common_dictionary = Dictionary(common_texts)

common_corpus = [common_dictionary.doc2bow(text) for text in common_texts]

# Train the model on the corpus.

lda = LdaModel(common_corpus, num_topics=10)

In [13]:
# 速度更快
from gensim.test.utils import common_corpus, common_dictionary
lda = LdaMulticore(common_corpus, id2word=common_dictionary, num_topics=10)

In [12]:
lda.print_topics(10)

[(0,
  '0.500*"trees" + 0.045*"system" + 0.045*"response" + 0.045*"graph" + 0.045*"minors" + 0.045*"interface" + 0.045*"user" + 0.045*"human" + 0.045*"time" + 0.045*"survey"'),
 (1,
  '0.083*"trees" + 0.083*"graph" + 0.083*"system" + 0.083*"user" + 0.083*"interface" + 0.083*"minors" + 0.083*"human" + 0.083*"time" + 0.083*"response" + 0.083*"computer"'),
 (2,
  '0.083*"trees" + 0.083*"system" + 0.083*"graph" + 0.083*"user" + 0.083*"minors" + 0.083*"interface" + 0.083*"survey" + 0.083*"response" + 0.083*"human" + 0.083*"computer"'),
 (3,
  '0.083*"trees" + 0.083*"graph" + 0.083*"system" + 0.083*"user" + 0.083*"interface" + 0.083*"time" + 0.083*"survey" + 0.083*"eps" + 0.083*"response" + 0.083*"human"'),
 (4,
  '0.262*"minors" + 0.262*"graph" + 0.262*"survey" + 0.024*"trees" + 0.024*"system" + 0.024*"user" + 0.024*"interface" + 0.024*"human" + 0.024*"computer" + 0.024*"response"'),
 (5,
  '0.153*"computer" + 0.153*"survey" + 0.153*"response" + 0.153*"system" + 0.153*"time" + 0.153*"user" 

# PLSA

In [3]:
from IPython.display import IFrame
IFrame('https://arxiv.org/pdf/1301.6705.pdf', width=950, height=450)

## Count Data and Co-occurrence Tables
文档集合：$\mathcal{D} = {d_1, d_2, ... , d_N}$\
构成文档的词汇表：$\mathcal{W} = {w_1, w_2, ... , w_M}$

term-document矩阵，也叫N×M共现计数表（忽略文档中词出现顺序）：$\mathbf{N}=(n(d_i, w_j))_{ij}$, 其中$n(d,w)$表示文档d中w出现的频次。

## The Aspect Model
![GraphicalMOdel](./assets/plsa_graphical_model.png)

d,w共现概率：$P(d,w) = P(d)P(w|d)$

引入潜在主题：$P(w|d) = \sum_{z \in Z }P(w|z)P(z|d)$

tips:
> **假设d与w独立**

于是,根据(b),d,w的联合概率：
$$
P(d,w) = \sum_{z \in Z } P(z)P(d|z)P(w|z)
$$
根据最上面2个式子，也等于：
$$
P(d,w) =\sum_{z \in Z }P(w|z)P(z|d)  P(d)
$$

## Model FItting with EM Algroithm

tips:EM算法推导见：[聚类(k-means,EM与高斯混合)](https://localhost:8888/lab/workspaces/auto-v/tree/mm_note.git/0010-%E6%9C%BA%E5%99%A8%E5%AD%A6%E4%B9%A0/%E7%BB%9F%E8%AE%A1%E6%9C%BA%E5%99%A8%E5%AD%A6%E4%B9%A0/0040-%E8%81%9A%E7%B1%BB(k-means%2CEM%E4%B8%8E%E9%AB%98%E6%96%AF%E6%B7%B7%E5%90%88%E8%81%9A%E7%B1%BB).ipynb)



包括共现模型与生成模型。上述论文为共现模型

### 建立似然函数
1） 语料生成概率：
$$
P(\mathcal{D}) = \prod_{i=1}^{N} \prod_{j=1}^{M} (P(d_i, w_j))^{n(d_i,w_j)} \ \ \ \ (1)
$$

2) 似然函数
$$
{\mathcal{L} = logP(\mathcal{D}) = log \prod_{i=1}^{N} \prod_{j=1}^{M} (P(d_i, w_j))^{n(d_i,w_j)} \\
= \sum_{i=1}^{N} \sum_{j=1}^{M} n(d_i, w_j)log P(d_i, w_j)\\
= \sum_{i=1}^{N} \sum_{j=1}^{M} n(d_i, w_j) log( \sum_{k=1}^{K} P(w_j|z_k)) P(z_k|d_i) P(d_i) \ \ \ (2)
}
$$

同时也等于：
$$
{\mathcal{L} = \sum_{i=1}^{N} \sum_{j=1}^{M} n(d_i, w_j)log(\sum_{k=1}^{K} P(w_j|z_k)) P(d_i|z_k) P(z_k) \ \ \ (3)
}
$$

对于（2）式子，可以进行如下拆分：
$$
{\mathcal{L} = \sum_{i=1}^{N} \sum_{j=1}^{M} n(d_i, w_j) log( \sum_{k=1}^{K} P(w_j|z_k)) P(z_k|d_i) P(d_i)\\
= \sum_{i=1}^{N} \sum_{j=1}^{M} n(d_i, w_j)log(P(d_i)\sum_{k=1}^{K} P(w_j|z_k)) P(z_k|d_i))\\
= \sum_{i=1}^{N} \sum_{j=1}^{M} n(d_i, w_j)log(P(d_i)) + \sum_{i=1}^{N} \sum_{j=1}^{M} n(d_i, w_j)log(\sum_{k=1}^{K} P(w_j|z_k)) P(z_k|d_i))
}
$$
因为前半段是常量，可以去掉，所以:
$$
{\mathcal{L} \propto \mathcal{{l}'} = \sum_{i=1}^{N} \sum_{j=1}^{M} n(d_i, w_j)log(\sum_{k=1}^{K} P(w_j|z_k)) P(z_k|d_i)) \\ 
= \sum_{i=1}^{N} \sum_{j=1}^{M} n(d_i, w_j)log(p(w_j|d_i))
} \ \ \ \ \ \ (4)
$$


### E Step:计算变量后验概率
论文中是共现模型，可以对式（3）用Jensen不等式，得到Q函数
$$
P(z|d,w) = \frac{P(z)P(d|z)P(w|z)}{\sum_{{z}' \in Z} P({z}')P(d|{z}')P(w|{z}')}
$$

**计算期望**：周志华《机器学习》-173页解释的简单清楚
此时求：$E({l}')$,因为$P(z_k|d_i)$与$P(w_j|z_k)$为概率分布（相当于约束），则引入拉格朗日乘子，得到新的似然函数。

1. 计算关于隐变量的后验概率（通过jensen不等式，得到Q函数就是基于隐变量的后验概率）
2. 根据后验概率，可以的到基于${l}'$的期望

此处，我们使用生成模型,根据jensen不等式，得到Q函数(也就是后验概率)：
$$
P(z_k|d_i, w_j) = \frac{P(w_j|z_k)P(d_i|z_k)}{\sum_{k=1}^{K}P(w_j|z_k)P(d_i|z_k)}
$$

此时初始化：$P(z_k|d_i)$与$P(w_j|z_k)$。通常要多初始化几组，防止陷入局部最优

### M Step：更新参数
极大化 $ \mathcal{{l}’} $

求导数，得到$P(z|d)$，$P(w|z)$公式，根据第一步结果，求解。

# Ref
[PLSA详解推导](https://blog.csdn.net/ustccyf/article/details/8957082)


https://my.oschina.net/u/3579120/blog/1508252 结合了论文，提到了$\beta$，需要系看一下

https://blog.csdn.net/hohaizx/article/details/88053429 推导也很详细

https://zhuanlan.zhihu.com/p/80557306 最清楚